In [ ]:
# ! pip install kaggle
import pandas as pd
from google.colab import drive
from torch.utils.tensorboard import SummaryWriter
drive.mount('/content/gdrive/')

%cd /content/gdrive/My Drive/Kaggle
# writer = SummaryWriter('')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/My Drive/Kaggle


In [ ]:
import random
!pip install transformers
import numpy as np
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, BertPreTrainedModel
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm, trange
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

     |████████████████████████████████| 3.1 MB 7.2 MB/s 
     |████████████████████████████████| 61 kB 671 kB/s 
     |████████████████████████████████| 895 kB 47.9 MB/s 
     |████████████████████████████████| 3.3 MB 46.7 MB/s 
     |████████████████████████████████| 596 kB 48.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
def set_seed():
    random.seed(77)
    np.random.seed(77)
    torch.manual_seed(77)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(77)

In [ ]:
#! unzip valid.tsv.zip
#! unzip train.tsv.zip
df = pd.read_csv('train.tsv', sep='\t')

In [ ]:
df_en = pd.read_csv('en.tsv', sep = '\t')
df_en

,Relation,Sentence,NER1,NER2
0,director,<e1>The Battle of the Sexes</e1> is a 1928 Ame...,WORK_OF_ART,PERSON
1,director,<e1>The Day of Faith</e1> is a 1923 American s...,WORK_OF_ART,PERSON
2,director,<e1>The Straight Story</e1> is a 1999 biograph...,WORK_OF_ART,PERSON
3,director,"Baltz starred in ""<e2>Paul Thomas Anderson</e2...",WORK_OF_ART,PERSON
4,director,He also appeared in <e2>Clint Eastwood</e2>'s ...,WORK_OF_ART,PERSON
...,...,...,...,...
3014,capital,He is a native of the town of <e2>Malolos</e2>...,GPE,GPE
3015,capital,<e2>Moscow</e2> is the capital of <e1>Russia</...,GPE,GPE
3016,capital,<e2>Seoul</e2> was the capital of various Kore...,GPE,GPE
3017,capital,The headquarters of the institution are locate...,GPE,GPE


In [ ]:
!pip install torch==1.3.1+cpu -f https://download.pytorch.org/whl/torch_stable.html
!pip install inltk

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 111.8 MB 80 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.3.1+cpu which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.3.1+cpu which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.3.1+cpu which is incompatible.


     |████████████████████████████████| 78 kB 3.2 MB/s 
     |████████████████████████████████| 233 kB 11.4 MB/s 
     |████████████████████████████████| 1.1 MB 52.9 MB/s 
     |████████████████████████████████| 1.2 MB 41.5 MB/s 
     |████████████████████████████████| 271 kB 42.2 MB/s 
     |████████████████████████████████| 160 kB 55.2 MB/s 
     |████████████████████████████████| 192 kB 45.1 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.3 MB/s eta 0:00:39tcmalloc: large alloc 1147494400 bytes == 0x55b82333a000 @  0x7fce4e542615 0x55b7e8ffd4cc 0x55b7e90dd47a 0x55b7e90002ed 0x55b7e90f1e1d 0x55b7e9073e99 0x55b7e906e9ee 0x55b7e9001bda 0x55b7e9073d00 0x55b7e906e9ee 0x55b7e9001bda 0x55b7e9070737 0x55b7e90f2c66 0x55b7e906fdaf 0x55b7e90f2c66 0x55b7e906fdaf 0x55b7e90f2c66 0x55b7e906fdaf 0x55b7e9002039 0x55b7e9045409 0x55b7e9000c52 0x55b7e9073c25 0x55b7e906e9ee 0x55b7e9001bda 0x55b7e9070737 0x55b7e906e9ee 0x55b7e9001bda 0x55b7e906f915 0x55b7e9001afa 0x55b7e906fc0d 0x55b7e906e9ee
  

In [ ]:
from inltk.inltk import identify_language, tokenize, reset_language_identifying_models
inputs = []

reset_language_identifying_models()
#identify_language('कहानियों में यह सुना जाता है कि एक रहस्यमय तिब्बती सिद्ध पुरुष और ')

for i in range(len(df)):
  reset_language_identifying_models()
  sentence = str(df['Sentence'].iloc[i])
  lang = identify_language(sentence)
  sentence = ' '.join(tokenize(sentence, lang))
  inputs.append(df['NER1'].iloc[i]+'-'+df['NER2'].iloc[i]+' '+sentence)
  break

df['CLS_sentence'] = inputs

df = df[['CLS_sentence', 'Relation']]
print(df)

FileNotFoundError: ignored

In [ ]:
ADDITIONAL_SPECIAL_TOKENS = ["<e1>", "</e1>", "<e2>", "</e2>"]
def load_tokenizer():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenizer.add_special_tokens({"additional_special_tokens": ADDITIONAL_SPECIAL_TOKENS})
    return tokenizer

In [ ]:
df['NER1'].unique()

array(['PERSON', 'EVENT', 'ORG', 'OTHER', 'GPE', 'WORK_OF_ART', 'LOC',
       'NORP', 'PRODUCT', 'FAC', 'GE', 'DATE'], dtype=object)

In [ ]:
df['NER2'].unique()

array(['OTHER', 'PERSON', 'ORG', 'GPE', 'LOC', 'LANGUAGE', 'NORP',
       'WORK_OF_ART', 'EVENT'], dtype=object)

In [ ]:
df['Relation'].unique()

array(['occupation', 'sport', 'sibling', 'parent_organization', 'mother',
       'father', 'place_of_birth', 'winner', 'student_of', 'employer',
       'spouse', 'founded_by', 'child', 'capital', 'director',
       'tributary', 'subsidiary', 'original_language_of_film_or_TV_show',
       'participant', 'award_received', 'place_of_death', 'capital_of',
       'position_held', 'discoverer_or_inventor', 'nominated_for'],
      dtype=object)

In [ ]:
labels = []
for i in range(len(df_en)):
  labels.append(df_en['NER1'].iloc[i]+'-'+df_en['NER2'].iloc[i])

df_en['CLS'] = labels

In [ ]:
df_en['CLS'].unique()

array(['WORK_OF_ART-PERSON', 'ORG-PERSON', 'PERSON-ORG', 'PERSON-NORP',
       'OTHER-PERSON', 'EVENT-GPE', 'PERSON-PERSON', 'ORG-ORG',
       'PERSON-OTHER', 'GPE-GPE', 'OTHER-GPE', 'OTHER-OTHER', 'LOC-LOC',
       'PERSON-GPE', 'WORK_OF_ART-LANGUAGE'], dtype=object)

In [ ]:
labels = list(set(labels))
id = {}
for i in range(len(labels)):
  id[labels[i]] = i

lst = []
for i in range(len(df)):
  lst.append(id[df['CLS']])
df['label_id'] = lst

KeyError: ignored

In [ ]:
def compute_metrics(preds, labels):
    assert len(preds) == len(labels)
    return acc_and_f1(preds, labels)

In [ ]:
def convert_inputs_to_features(
    texts,
    labels,
    max_seq_len,
    tokenizer,
    cls_token="[CLS]",
    sep_token="[SEP]",
    pad_token=0,
    mask_padding_with_zero=True,
):
    features = []
    for (ex_index, example) in enumerate(texts):

        tokens_a = tokenizer.tokenize(example)

        e11_p = tokens_a.index("<e1>")  # the start position of entity1
        e12_p = tokens_a.index("</e1>")  # the end position of entity1
        e21_p = tokens_a.index("<e2>")  # the start position of entity2
        e22_p = tokens_a.index("</e2>")  # the end position of entity2

        # Replace the token
        tokens_a[e11_p] = "$"
        tokens_a[e12_p] = "$"
        tokens_a[e21_p] = "#"
        tokens_a[e22_p] = "#"

        # Add 1 because of the [CLS] token
        e11_p += 1
        e12_p += 1
        e21_p += 1
        e22_p += 1

        # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
        special_tokens_count = 2
        if len(tokens_a) > max_seq_len - special_tokens_count:
            tokens_a = tokens_a[: (max_seq_len - special_tokens_count)]

        tokens = [cls_token] + tokens_a + [sep_token]

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real tokens are attended to.
        attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding_length = max_seq_len - len(input_ids)
        input_ids = input_ids + ([pad_token] * padding_length)
        attention_mask = attention_mask + ([0 if mask_padding_with_zero else 1] * padding_length)

        # e1 mask, e2 mask
        e1_mask = [0] * len(attention_mask)
        e2_mask = [0] * len(attention_mask)

        for i in range(e11_p, e12_p + 1):
            e1_mask[i] = 1
        for i in range(e21_p, e22_p + 1):
            e2_mask[i] = 1

        assert len(input_ids) == max_seq_len, "Error with input length {} vs {}".format(len(input_ids), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(
            len(attention_mask), max_seq_len
        )
        assert len(token_type_ids) == max_seq_len, "Error with token type length {} vs {}".format(
            len(token_type_ids), max_seq_len
        )

        label_id = int(labels)

        features.append(
            InputFeatures(
                input_ids=input_ids,
                attention_mask=attention_mask,
                label_id=label_id,
                e1_mask=e1_mask,
                e2_mask=e2_mask,
            )
        )

    return features


In [ ]:
def load_and_cache_examples(tokenizer, mode):
    features = convert_examples_to_features(
            df_en.Sentence.tolist(), df_en.CLS.tolist(), 384, tokenizer, add_sep_token=True
        )
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    all_e1_mask = torch.tensor([f.e1_mask for f in features], dtype=torch.long)  # add e1 mask
    all_e2_mask = torch.tensor([f.e2_mask for f in features], dtype=torch.long)  # add e2 mask

    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)

    dataset = TensorDataset(
        all_input_ids,
        all_attention_mask,
        all_token_type_ids,
        all_label_ids,
        all_e1_mask,
        all_e2_mask,
    )
    return dataset

In [ ]:
class FCLayer(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.0, use_activation=True):
        super(FCLayer, self).__init__()
        self.use_activation = use_activation
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(input_dim, output_dim)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.dropout(x)
        if self.use_activation:
            x = self.tanh(x)
        return self.linear(x)


class RBERT(BertPreTrainedModel):
    def __init__(self, config):
        super(RBERT, self).__init__(config)
        self.bert = BertModel(config=config)  # Load pretrained bert

        self.num_labels = config.num_labels

        self.cls_fc_layer = FCLayer(config.hidden_size, config.hidden_size, 0.1) # dropout_rate = 0.1
        self.entity_fc_layer = FCLayer(config.hidden_size, config.hidden_size, 0.1)
        self.label_classifier = FCLayer(
            config.hidden_size * 3,
            config.num_labels,
            0.1,
            use_activation=False,
        )

    @staticmethod
    def entity_average(hidden_output, e_mask):
        """
        Average the entity hidden state vectors (H_i ~ H_j)
        :param hidden_output: [batch_size, j-i+1, dim]
        :param e_mask: [batch_size, max_seq_len]
                e.g. e_mask[0] == [0, 0, 0, 1, 1, 1, 0, 0, ... 0]
        :return: [batch_size, dim]
        """
        e_mask_unsqueeze = e_mask.unsqueeze(1)  # [b, 1, j-i+1]
        length_tensor = (e_mask != 0).sum(dim=1).unsqueeze(1)  # [batch_size, 1]

        # [b, 1, j-i+1] * [b, j-i+1, dim] = [b, 1, dim] -> [b, dim]
        sum_vector = torch.bmm(e_mask_unsqueeze.float(), hidden_output).squeeze(1)
        avg_vector = sum_vector.float() / length_tensor.float()  # broadcasting
        return avg_vector

    def forward(self, input_ids, attention_mask, token_type_ids, labels, e1_mask, e2_mask):
        outputs = self.bert(
            input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids
        )  # sequence_output, pooled_output, (hidden_states), (attentions)
        sequence_output = outputs[0]
        pooled_output = outputs[1]  # [CLS]

        # Average
        e1_h = self.entity_average(sequence_output, e1_mask)
        e2_h = self.entity_average(sequence_output, e2_mask)

        # Dropout -> tanh -> fc_layer (Share FC layer for e1 and e2)
        pooled_output = self.cls_fc_layer(pooled_output)
        e1_h = self.entity_fc_layer(e1_h)
        e2_h = self.entity_fc_layer(e2_h)

        # Concat -> fc_layer
        concat_h = torch.cat([pooled_output, e1_h, e2_h], dim=-1)
        logits = self.label_classifier(concat_h)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        # Softmax
        if labels is not None:
            if self.num_labels == 1:
                loss_fct = nn.MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [ ]:
class Trainer(object):
    def __init__(self, train_dataset=None, dev_dataset=None, test_dataset=None, num_train_epochs=10, weight_decay=0.0, adam_epsilon=1e-8, learning_rate=0.001, model_dir = './model'):
        #self.args = args
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        self.test_dataset = test_dataset

        self.label_lst = labels
        self.num_labels = len(self.label_lst)

        self.config = BertConfig.from_pretrained(
            'bert-base-uncased',
            num_labels=self.num_labels,
            finetuning_task='semeval',
            id2label={str(i): label for i, label in enumerate(self.label_lst)},
            label2id={label: i for i, label in enumerate(self.label_lst)},
        )
        self.model = RBERT.from_pretrained('bert-base-uncased', config=self.config)

        # GPU or CPU
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)

    def train(self):
        train_sampler = RandomSampler(self.train_dataset)
        train_dataloader = DataLoader(
            self.train_dataset,
            sampler=train_sampler,
            batch_size=16,
        )

        max_steps = -1
        gradient_accumulation_steps = 1

        if max_steps > 0:
            t_total = max_steps
            self.num_train_epochs = (
                max_steps // (len(train_dataloader) // gradient_accumulation_steps) + 1
            )
        else:
            t_total = len(train_dataloader) // gradient_accumulation_steps * self.num_train_epochs

        # Prepare optimizer and schedule (linear warmup and decay)
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.weight_decay,
            },
            {
                "params": [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=self.learning_rate,
            eps=self.adam_epsilon,
        )
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=0,
            num_training_steps=t_total,
        )

        # Train!
        print("***** Running training *****")
        print("  Num examples = %d", len(self.train_dataset))
        print("  Num Epochs = %d", 10)
        print("  Total train batch size = %d", 16)
        print("  Gradient Accumulation steps = %d", gradient_accumulation_steps)
        print("  Total optimization steps = %d", t_total)
        #print("  Logging steps = %d", self.args.logging_steps)
        print("  Save steps = %d", 250)

        global_step = 0
        tr_loss = 0.0
        self.model.zero_grad()

        train_iterator = trange(int(16), desc="Epoch")

        for _ in train_iterator:
            epoch_iterator = tqdm(train_dataloader, desc="Iteration")
            for step, batch in enumerate(epoch_iterator):
                self.model.train()
                batch = tuple(t.to(self.device) for t in batch)  # GPU or CPU
                inputs = {
                    "input_ids": batch[0],
                    "attention_mask": batch[1],
                    "token_type_ids": batch[2],
                    "labels": batch[3],
                    "e1_mask": batch[4],
                    "e2_mask": batch[5],
                }
                outputs = self.model(**inputs)
                loss = outputs[0]

                if gradient_accumulation_steps > 1:
                    loss = loss / gradient_accumulation_steps

                loss.backward()

                max_grad_norm = 1

                tr_loss += loss.item()
                if (step + 1) % gradient_accumulation_steps == 0:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_grad_norm)

                    optimizer.step()
                    scheduler.step()  # Update learning rate schedule
                    self.model.zero_grad()
                    global_step += 1

                    '''
                    if self.args.logging_steps > 0 and global_step % self.args.logging_steps == 0:
                        self.evaluate("test")  # There is no dev set for semeval task

                    if self.args.save_steps > 0 and global_step % self.args.save_steps == 0:
                        self.save_model()
                    '''

                if 0 < max_steps < global_step:
                    epoch_iterator.close()
                    break

            if 0 < max_steps < global_step:
                train_iterator.close()
                break

        return global_step, tr_loss / global_step

    def evaluate(self, mode):
        # We use test dataset because semeval doesn't have dev dataset
        if mode == "test":
            dataset = self.test_dataset
        elif mode == "dev":
            dataset = self.dev_dataset
        else:
            raise Exception("Only dev and test dataset available")

        eval_sampler = SequentialSampler(dataset)
        eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=32)

        # Eval!
        #print("***** Running evaluation on %s dataset *****", mode)
        #print("  Num examples = %d", len(dataset))
        #print("  Batch size = %d", self.args.eval_batch_size)
        eval_loss = 0.0
        nb_eval_steps = 0
        preds = None
        out_label_ids = None

        self.model.eval()

        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            batch = tuple(t.to(self.device) for t in batch)
            with torch.no_grad():
                inputs = {
                    "input_ids": batch[0],
                    "attention_mask": batch[1],
                    "token_type_ids": batch[2],
                    "labels": batch[3],
                    "e1_mask": batch[4],
                    "e2_mask": batch[5],
                }
                outputs = self.model(**inputs)
                tmp_eval_loss, logits = outputs[:2]

                eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1

            if preds is None:
                preds = logits.detach().cpu().numpy()
                out_label_ids = inputs["labels"].detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

        eval_loss = eval_loss / nb_eval_steps
        results = {"loss": eval_loss}
        preds = np.argmax(preds, axis=1)
        #write_prediction(self.args, os.path.join(self.args.eval_dir, "proposed_answers.txt"), preds)

        result = compute_metrics(preds, out_label_ids)
        results.update(result)

        for key in sorted(results.keys()):
            print("  {} = {:.4f}".format(key, results[key]))

        return results

    '''
    def save_model(self):
        # Save model checkpoint (Overwrite)
        if not os.path.exists(self.model_dir):
            os.makedirs(self.model_dir)
        model_to_save = self.model.module if hasattr(self.model, "module") else self.model
        model_to_save.save_pretrained(self.model_dir)

        # Save training arguments together with the trained model
        torch.save(self.args, os.path.join(self.model_dir, "training_args.bin"))
        print("Saving model checkpoint to %s", self.model_dir)

    def load_model(self):
        # Check whether model exists
        if not os.path.exists(self.model_dir):
            raise Exception("Model doesn't exists! Train first!")

        self.args = torch.load(os.path.join(self.model_dir, "training_args.bin"))
        self.model = RBERT.from_pretrained(self.model_dir)
        self.model.to(self.device)
        print("***** Model Loaded *****")
        '''

In [ ]:
num_train_epochs=10
weight_decay=0.0
adam_epsilon=1e-8
learning_rate=0.001

set_seed()
tokenizer = load_tokenizer()

train_dataset = load_and_cache_examples(tokenizer, mode="train")
test_dataset = load_and_cache_examples(tokenizer, mode="test")

trainer = Trainer(train_dataset, test_dataset, num_train_epochs, weight_decay, adam_epsilon, learning_rate)

'''
if args.do_train:
    trainer.train()

if args.do_eval:
    trainer.load_model()
    trainer.evaluate("test")
'''

TypeError: ignored